# Process bulk counts table

1. Annotate columns (samples) in a way that time point and replicate are annotated
2. Remove rRNA genes
3. Perform TPM normalization and alternatively also raw counts

In [1]:
# Use miniconda environment Jupyter_new for running this notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from tools import *
import warnings
warnings.filterwarnings('ignore')

## 1 Load the dataset and annotation

In [2]:
bulkPath = '../nf_output/countData/countData.tsv'
metaPath = '../SraRunTable.csv' # metadata from SRA
gffPath = '../nf_output/alignments/dualGenome.gff3' # output from nf pipeline

In [3]:
# Load and filter data
df_initial = pd.read_csv(bulkPath, sep = '\t', comment='#', index_col=0)
df_initial = df_initial.drop(columns=['SRR11805719_sorted.bam', 'SRR11805717_sorted.bam', 'SRR11805716_sorted.bam',
                                      'SRR11805718_sorted.bam', 'SRR11805715_sorted.bam'])
metadata = pd.read_csv(metaPath)
metadata = metadata[metadata['infection'] == 'uninfected control']

In [4]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 20)
metadata

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,DATASTORE provider,DATASTORE region,Experiment,GEO_Accession (exp),infection,Instrument,LibraryLayout,LibrarySelection,LibrarySource,Organism,Platform,ReleaseDate,create_date,version,Sample Name,source_name,SRA Study,time,HOST
5,SRR11805720,RNA-Seq,300,2674104900,PRJNA633474,SAMN14944074,1111740814,GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",SRX8357152,GSM4557362,uninfected control,Illumina HiSeq 4000,PAIRED,cDNA,TRANSCRIPTOMIC,Synechococcus sp. WH 7803,ILLUMINA,2020-12-07T00:00:00Z,2020-05-18T08:28:00Z,1,GSM4557362,Synechococcus sp. strain WH7803,SRP262107,30 min,NaN
6,SRR11805721,RNA-Seq,300,3106973400,PRJNA633474,SAMN14944073,1264730669,GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",SRX8357153,GSM4557363,uninfected control,Illumina HiSeq 4000,PAIRED,cDNA,TRANSCRIPTOMIC,Synechococcus sp. WH 7803,ILLUMINA,2020-12-07T00:00:00Z,2020-05-18T08:30:00Z,1,GSM4557363,Synechococcus sp. strain WH7803,SRP262107,8 h,NaN


## 2 Format the dataset

### 2.1 Annotate sample names

Issue now is that the metadata does not properly annotate sample names. Thus, this will be done manually by adding another sample name column to the metadata.

In [5]:
# Match GSM IDs and SampleNames inferred from GEO

sampleDict = {'GSM4557362': '30_R1', 'GSM4557363': '480_R1'
}

In [6]:
metadataFull = annotateData(metadata, sampleDict)

In [7]:
metadata.head(1)

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,DATASTORE provider,DATASTORE region,Experiment,GEO_Accession (exp),infection,Instrument,LibraryLayout,LibrarySelection,LibrarySource,Organism,Platform,ReleaseDate,create_date,version,Sample Name,source_name,SRA Study,time,HOST
5,SRR11805720,RNA-Seq,300,2674104900,PRJNA633474,SAMN14944074,1111740814,GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",SRX8357152,GSM4557362,uninfected control,Illumina HiSeq 4000,PAIRED,cDNA,TRANSCRIPTOMIC,Synechococcus sp. WH 7803,ILLUMINA,2020-12-07T00:00:00Z,2020-05-18T08:28:00Z,1,GSM4557362,Synechococcus sp. strain WH7803,SRP262107,30 min,NaN


In [8]:
metadataFull.head()

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,DATASTORE provider,DATASTORE region,Experiment,GEO_Accession (exp),infection,Instrument,LibraryLayout,LibrarySelection,LibrarySource,Organism,Platform,ReleaseDate,create_date,version,Sample Name,source_name,SRA Study,time,HOST,SampleID,SampleNames
SampleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SRR11805720_sorted.bam,SRR11805720,RNA-Seq,300,2674104900,PRJNA633474,SAMN14944074,1111740814,GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",SRX8357152,GSM4557362,uninfected control,Illumina HiSeq 4000,PAIRED,cDNA,TRANSCRIPTOMIC,Synechococcus sp. WH 7803,ILLUMINA,2020-12-07T00:00:00Z,2020-05-18T08:28:00Z,1,GSM4557362,Synechococcus sp. strain WH7803,SRP262107,30 min,NaN,SRR11805720_sorted.bam,30_R1
SRR11805721_sorted.bam,SRR11805721,RNA-Seq,300,3106973400,PRJNA633474,SAMN14944073,1264730669,GEO,public,"fastq,run.zq,sra","gs,ncbi,s3","gs.us-east1,ncbi.public,s3.us-east-1",SRX8357153,GSM4557363,uninfected control,Illumina HiSeq 4000,PAIRED,cDNA,TRANSCRIPTOMIC,Synechococcus sp. WH 7803,ILLUMINA,2020-12-07T00:00:00Z,2020-05-18T08:30:00Z,1,GSM4557363,Synechococcus sp. strain WH7803,SRP262107,8 h,NaN,SRR11805721_sorted.bam,480_R1


Add correct sample names.

In [9]:
df_initial

,Chr,Start,End,Strand,Length,SRR11805720_sorted.bam,SRR11805721_sorted.bam
Geneid,,,,,,,
gene-SynWH7803_0001,CT971583.1,174,1343,+,1170,1041,822
gene-SynWH7803_0002,CT971583.1,1347,2096,+,750,301,233
gene-SynWH7803_0003,CT971583.1,2187,4484,+,2298,369,464
gene-SynWH7803_0004,CT971583.1,4523,5989,+,1467,626,605
gene-SynWH7803_0005,CT971583.1,5992,8457,-,2466,1154,839
...,...,...,...,...,...,...,...
gene-SSBP1_gp51,MT424636.1,41158,41403,+,246,4,41
gene-SSBP1_gp52,MT424636.1,41390,41647,+,258,2,42
gene-SSBP1_gp53,MT424636.1,41974,42744,+,771,20,92


In [10]:
df = changeColnames(df_initial.iloc[:,5:df_initial.shape[1]], metadataFull)
df = df[['30_R1', '480_R1']]
df.head()

SampleNames,30_R1,480_R1
Geneid,,
gene-SynWH7803_0001,1041,822
gene-SynWH7803_0002,301,233
gene-SynWH7803_0003,369,464
gene-SynWH7803_0004,626,605
gene-SynWH7803_0005,1154,839


### 2.2 Remove rRNA genes

Consult gff3 file to get rRNA geneids.

In [11]:
# Load gff3 and split into genes and CDS dfs
gff3 = pd.read_csv(gffPath, sep='\t', header = None, skiprows = 5)
gff3.columns=["seq_id", "source", "type", "start", "end", "phase", "strand", "score", "attributes"]
gff3_genes = gff3.loc[gff3["type"] == 'gene']

# Column formating for genes
gff3_genes = gff3_genes.reset_index(drop=True)
dct_genes = gff3_genes["attributes"].str.split(';').apply(lambda items: dict(item.split('=', 1) for item in items if '=' in item))
cols_to_keep = ['ID', 'Name', 'gbkey', 'gene_biotype', 'locus_tag', 'gene']
gff3_genes = pd.concat([gff3_genes, pd.json_normalize(dct_genes)[cols_to_keep]], axis=1)

# Generate locus_tag, product dictonary over all different feature types
attrs = gff3["attributes"].str.split(";", expand=True)
attrs_dicts = attrs.apply(lambda row: {item.split("=")[0]: item.split("=")[1] for item in row if "=" in str(item)}, axis=1)
attrs_df = pd.json_normalize(attrs_dicts)
attrs_df = attrs_df.dropna(subset=["locus_tag", "product"])
locus_product_dict = dict(zip(attrs_df["locus_tag"], attrs_df["product"]))

# Add gene product, if not stated in gff3, fill with gene_biotype
gff3_genes["product"] = gff3_genes["locus_tag"].map(locus_product_dict)
# gff3_genes["product"] = gff3_genes["product"].fillna("other")
gff3_genes["product"] = gff3_genes["product"].fillna(gff3_genes["gene_biotype"])

# If gene = NA, take from ID column
gff3_genes["gene"] = gff3_genes["gene"].fillna(gff3_genes["ID"])

# Drop attributes column
gff3_genes = gff3_genes.drop(["attributes"], axis=1)

In [12]:
pharokka_path = "../../../2025-12_reannotation_phage_genomes/Pharokka_proteins_phages_out/MT424636.1_out/pharokka_proteins_full_merged_output.tsv"
gff3_genes = add_pharokka(gff3_genes, pharokka_path)
gff3_genes.loc[gff3_genes['seq_id'] == "MT424636.1"]

,seq_id,source,type,start,end,phase,strand,score,ID,Name,gbkey,gene_biotype,locus_tag,gene,product,annot,PHROG,category
2586,MT424636.1,Genbank,gene,649.0,924.0,.,+,.,gene-SSBP1_gp01,SSBP1_gp01,Gene,protein_coding,SSBP1_gp01,gene-SSBP1_gp01,hypothetical protein,hypothetical protein,7365,unknown function
2587,MT424636.1,Genbank,gene,1019.0,1330.0,.,+,.,gene-SSBP1_gp02,SSBP1_gp02,Gene,protein_coding,SSBP1_gp02,gene-SSBP1_gp02,hypothetical protein,hypothetical protein,2543,unknown function
2588,MT424636.1,Genbank,gene,1405.0,1566.0,.,+,.,gene-SSBP1_gp03,SSBP1_gp03,Gene,protein_coding,SSBP1_gp03,gene-SSBP1_gp03,hypothetical protein,hypothetical protein,No_PHROG,unknown function
2589,MT424636.1,Genbank,gene,1566.0,1910.0,.,+,.,gene-SSBP1_gp04,SSBP1_gp04,Gene,protein_coding,SSBP1_gp04,gene-SSBP1_gp04,hypothetical protein,hypothetical protein,554,unknown function
2590,MT424636.1,Genbank,gene,1907.0,2095.0,.,+,.,gene-SSBP1_gp05,SSBP1_gp05,Gene,protein_coding,SSBP1_gp05,gene-SSBP1_gp05,hypothetical protein,hypothetical protein,No_PHROG,unknown function
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636,MT424636.1,Genbank,gene,41158.0,41403.0,.,+,.,gene-SSBP1_gp51,SSBP1_gp51,Gene,protein_coding,SSBP1_gp51,gene-SSBP1_gp51,hypothetical protein,terminase small subunit,891,head and packaging
2637,MT424636.1,Genbank,gene,41390.0,41647.0,.,+,.,gene-SSBP1_gp52,SSBP1_gp52,Gene,protein_coding,SSBP1_gp52,gene-SSBP1_gp52,hypothetical protein,hypothetical protein,6437,unknown function
2638,MT424636.1,Genbank,gene,41974.0,42744.0,.,+,.,gene-SSBP1_gp53,SSBP1_gp53,Gene,protein_coding,SSBP1_gp53,gene-SSBP1_gp53,hypothetical protein,hypothetical protein,No_PHROG,unknown function
2639,MT424636.1,Genbank,gene,42741.0,44471.0,.,+,.,gene-SSBP1_gp54,SSBP1_gp54,Gene,protein_coding,SSBP1_gp54,gene-SSBP1_gp54,terminase large subunit,terminase large subunit,17672,head and packaging


In [13]:
# Load ggf3 file

gff3 = pd.read_csv(gffPath, sep='\t', header = None, skiprows = 5)
gff3 = gff3.loc[gff3.iloc[:,2] == 'gene']

# Format some new columns
gff3['ID'] = pd.DataFrame(gff3.iloc[:,8].str.split('ID=', expand = True)).iloc[:,1].str.split(';', expand = True).iloc[:,0]
gff3['GeneType'] = pd.DataFrame(gff3.iloc[:,8].str.split('gene_biotype=', expand = True)).iloc[:,1].str.split(';', expand = True).iloc[:,0]
gff3['Symbol'] = pd.DataFrame(gff3.iloc[:,8].str.split('gene=', expand = True)).iloc[:,1].str.split(';', expand = True).iloc[:,0]

# Add entity host and phage
gff3['Entity'] = np.where(gff3[0] == 'CT971583.1', 'host', 'phage')
gff3.index = gff3['ID']
rRNAs = gff3.loc[gff3['GeneType'] == 'rRNA', 'ID'].tolist()

In [14]:
gff3

,0,1,2,3,4,5,6,7,8,ID,GeneType,Symbol,Entity
ID,,,,,,,,,,,,,
gene-SynWH7803_0001,CT971583.1,EMBL,gene,174.0,1343.0,.,+,.,ID=gene-SynWH7803_0001;Name=dnaN;gbkey=Gene;ge...,gene-SynWH7803_0001,protein_coding,dnaN,host
gene-SynWH7803_0002,CT971583.1,EMBL,gene,1347.0,2096.0,.,+,.,ID=gene-SynWH7803_0002;Name=SynWH7803_0002;gbk...,gene-SynWH7803_0002,protein_coding,None,host
gene-SynWH7803_0003,CT971583.1,EMBL,gene,2187.0,4484.0,.,+,.,ID=gene-SynWH7803_0003;Name=purL;gbkey=Gene;ge...,gene-SynWH7803_0003,protein_coding,purL,host
gene-SynWH7803_0004,CT971583.1,EMBL,gene,4523.0,5989.0,.,+,.,ID=gene-SynWH7803_0004;Name=purF;gbkey=Gene;ge...,gene-SynWH7803_0004,protein_coding,purF,host
gene-SynWH7803_0005,CT971583.1,EMBL,gene,5992.0,8457.0,.,-,.,ID=gene-SynWH7803_0005;Name=gyrA;gbkey=Gene;ge...,gene-SynWH7803_0005,protein_coding,gyrA,host
...,...,...,...,...,...,...,...,...,...,...,...,...,...
gene-SSBP1_gp51,MT424636.1,Genbank,gene,41158.0,41403.0,.,+,.,ID=gene-SSBP1_gp51;Name=SSBP1_gp51;gbkey=Gene;...,gene-SSBP1_gp51,protein_coding,None,phage
gene-SSBP1_gp52,MT424636.1,Genbank,gene,41390.0,41647.0,.,+,.,ID=gene-SSBP1_gp52;Name=SSBP1_gp52;gbkey=Gene;...,gene-SSBP1_gp52,protein_coding,None,phage
gene-SSBP1_gp53,MT424636.1,Genbank,gene,41974.0,42744.0,.,+,.,ID=gene-SSBP1_gp53;Name=SSBP1_gp53;gbkey=Gene;...,gene-SSBP1_gp53,protein_coding,None,phage


Perform in silico rRNA depletion.

In [15]:
df_norRNAs = rRNAdepletion(df,rRNAs)
df_norRNAs.head()

SampleNames,30_R1,480_R1
Geneid,,
gene-SynWH7803_0646,139,152
gene-SynWH7803_1173,500,321
gene-SynWH7803_2450,253,290
gene-SynWH7803_0497,965,618
gene-SynWH7803_0475,153,157


### 2.3 Read count normalization

Important note: gene symbols not available for most genes.

In [16]:
# Function to fill in missing symbols by geneid.

def fillSymbols(df):
    df_new = df.copy()
    index = df.index.to_list()
    for i in range(0,df.shape[0]):
        if (df.iloc[i,-1:].values == None):
            df_new.iloc[i,-1:] = index[i]
    return df_new

Convert counts to TPM.

In [17]:
tpms = TPM(df_norRNAs, df_initial, 0.5)
tpms['Entity'] = gff3.loc[sorted(tpms.index.to_list()), 'Entity']
tpms['Symbol'] = gff3.loc[sorted(tpms.index.to_list()), 'Symbol']

tpms = fillSymbols(tpms)
tpms = make_unique_with_index(tpms)
tpms

SampleNames,30_R1,480_R1,Entity,Symbol
Geneid,,,,
gene-SynWH7803_0646,105.179205,134.082216,host,gene-SynWH7803_0646
gene-SynWH7803_1173,125.436432,93.960701,host,gene-SynWH7803_1173
gene-SynWH7803_2450,92.083888,123.054494,host,gene-SynWH7803_2450
gene-SynWH7803_0497,841.042013,628.276443,host,cpeR
gene-SynWH7803_0475,86.077775,102.993276,host,gene-SynWH7803_0475
...,...,...,...,...
gene-SynWH7803_1801,381.062213,413.929476,host,lepA
gene-SynWH7803_1597,36.084155,43.443403,host,gene-SynWH7803_1597
gene-SynWH7803_1516,117.919220,123.454206,host,gene-SynWH7803_1516


In [18]:
# Check gene names unique
len(tpms['Symbol'].unique())

2635

Log2+1 normalization raw counts

In [19]:
logs = logNorm(df_norRNAs)
logs['Entity'] = gff3.loc[sorted(logs.index.to_list()), 'Entity']
logs['Symbol'] = gff3.loc[sorted(logs.index.to_list()), 'Symbol']
logs = fillSymbols(logs)
# Make gene names unique
logs = make_unique_with_index(logs)
logs

SampleNames,30_R1,480_R1,Entity,Symbol
Geneid,,,,
gene-SynWH7803_0646,7.129283,7.257388,host,gene-SynWH7803_0646
gene-SynWH7803_1173,8.968667,8.330917,host,gene-SynWH7803_1173
gene-SynWH7803_2450,7.988685,8.184875,host,gene-SynWH7803_2450
gene-SynWH7803_0497,9.915879,9.273796,host,cpeR
gene-SynWH7803_0475,7.266787,7.303781,host,gene-SynWH7803_0475
...,...,...,...,...
gene-SynWH7803_1801,11.327553,11.225207,host,lepA
gene-SynWH7803_1597,6.539159,6.584963,host,gene-SynWH7803_1597
gene-SynWH7803_1516,9.631177,9.475733,host,gene-SynWH7803_1516


Log2+1 normalization tpms

In [20]:
logTPMs = logNorm(tpms.iloc[:, :-2])
logTPMs = logTPMs.join(tpms.iloc[:, -2:])
logTPMs = fillSymbols(logTPMs)
# Make gene names unique
logTPMs = make_unique_with_index(logTPMs)
logTPMs

SampleNames,30_R1,480_R1,Entity,Symbol
Geneid,,,,
gene-SynWH7803_0646,6.730357,7.077694,host,gene-SynWH7803_0646
gene-SynWH7803_1173,6.982268,6.569259,host,gene-SynWH7803_1173
gene-SynWH7803_2450,6.540460,6.954830,host,gene-SynWH7803_2450
gene-SynWH7803_0497,9.717748,9.297550,host,cpeR
gene-SynWH7803_0475,6.444233,6.700346,host,gene-SynWH7803_0475
...,...,...,...,...
gene-SynWH7803_1801,8.577664,8.696722,host,lepA
gene-SynWH7803_1597,5.212731,5.473897,host,gene-SynWH7803_1597
gene-SynWH7803_1516,6.893838,6.959471,host,gene-SynWH7803_1516


## 3 Filter samples, if necessary

One replicate, not necessary.

## 4. Final grouping

Summarize time points with mean and standard deviation for TPM-normalized data.

In [21]:
columnOrder = ['30_R1', '480_R1']

In [22]:
TPMmeans, TPMsds = getMeanSD(tpms[columnOrder])
TPMmeans = TPMmeans[['30', '480']]
TPMmeans[['Entity', 'Symbol']] = tpms[['Entity', 'Symbol']]
TPMmeans

,30,480,Entity,Symbol
Geneid,,,,
gene-SynWH7803_0646,105.179205,134.082216,host,gene-SynWH7803_0646
gene-SynWH7803_1173,125.436432,93.960701,host,gene-SynWH7803_1173
gene-SynWH7803_2450,92.083888,123.054494,host,gene-SynWH7803_2450
gene-SynWH7803_0497,841.042013,628.276443,host,cpeR
gene-SynWH7803_0475,86.077775,102.993276,host,gene-SynWH7803_0475
...,...,...,...,...
gene-SynWH7803_1801,381.062213,413.929476,host,lepA
gene-SynWH7803_1597,36.084155,43.443403,host,gene-SynWH7803_1597
gene-SynWH7803_1516,117.919220,123.454206,host,gene-SynWH7803_1516


In [23]:
TPMsds = TPMsds[['30', '480']]
TPMsds[['Entity', 'Symbol']] = tpms[['Entity', 'Symbol']]
TPMsds

,30,480,Entity,Symbol
Geneid,,,,
gene-SynWH7803_0646,0.0,0.0,host,gene-SynWH7803_0646
gene-SynWH7803_1173,0.0,0.0,host,gene-SynWH7803_1173
gene-SynWH7803_2450,0.0,0.0,host,gene-SynWH7803_2450
gene-SynWH7803_0497,0.0,0.0,host,cpeR
gene-SynWH7803_0475,0.0,0.0,host,gene-SynWH7803_0475
...,...,...,...,...
gene-SynWH7803_1801,0.0,0.0,host,lepA
gene-SynWH7803_1597,0.0,0.0,host,gene-SynWH7803_1597
gene-SynWH7803_1516,0.0,0.0,host,gene-SynWH7803_1516


In [24]:
propExp = proportionalExp(TPMmeans[['30', '480']])
propExp[['Entity', 'Symbol']] = TPMmeans[['Entity', 'Symbol']]
propExp

,30,480,Entity,Symbol
Geneid,,,,
gene-SynWH7803_0646,0.784438,1.000000,host,gene-SynWH7803_0646
gene-SynWH7803_1173,1.000000,0.749070,host,gene-SynWH7803_1173
gene-SynWH7803_2450,0.748318,1.000000,host,gene-SynWH7803_2450
gene-SynWH7803_0497,1.000000,0.747021,host,cpeR
gene-SynWH7803_0475,0.835761,1.000000,host,gene-SynWH7803_0475
...,...,...,...,...
gene-SynWH7803_1801,0.920597,1.000000,host,lepA
gene-SynWH7803_1597,0.830601,1.000000,host,gene-SynWH7803_1597
gene-SynWH7803_1516,0.955166,1.000000,host,gene-SynWH7803_1516


## 5. Phage gene classification

In [25]:
# Add a classification label based on exceeding 20 % of maximal expression

def classLabelThreshold(tpm):
    
    labels = list()
    
    i = 0
    while i < tpm.shape[0]:

        # Get array of expression values at time points
        expressions = list(tpm.iloc[i,0:(tpm.shape[1]-2)])

        # Get maximal value for each gene across time points
        maxTPM = max(expressions)

        # Get the threshold value
        thresHold = maxTPM*0.2

        # Subset expressions based on threshold
        filteredExpressions = [x for x in expressions if x >= thresHold]

        # Get index of time point
        indices = [expressions.index(x) for x in filteredExpressions]
        timePoint = min(indices)

        if timePoint == 0:
            labels.append('early')
        elif timePoint == 1:
            labels.append('early')
        elif timePoint == 2:
            labels.append('early')
        elif timePoint == 3:
            labels.append('middle')
        elif timePoint == 4:
            labels.append('late')
        i += 1

    tpmOut = tpm.copy()
    tpmOut['ClassThreshold'] = labels
    tpmOut.loc[tpmOut['Entity'] == 'host', 'ClassThreshold'] = 'None'

    return tpmOut

In [26]:
# Add a classification label based on exceeding 20 % of maximal expression

def classLabelMax(tpm):
    
    labels = list()
    
    i = 0
    while i < tpm.shape[0]:

        # Get array of expression values at time points
        expressions = list(tpm.iloc[i,0:(tpm.shape[1]-3)])

        # Get maximal value for each gene across time points
        maxTPM = max(expressions)

        # Get the threshold value
        thresHold = maxTPM

        # Subset expressions based on threshold
        filteredExpressions = [x for x in expressions if x == thresHold]

        # Get index of time point
        indices = [expressions.index(x) for x in filteredExpressions]
        timePoint = min(indices)

        if timePoint == 0:
            labels.append('early')
        elif timePoint == 1:
            labels.append('early')
        elif timePoint == 2:
            labels.append('early')
        elif timePoint == 3:
            labels.append('middle')
        elif timePoint == 4:
            labels.append('late')

        i += 1

    tpmOut = tpm.copy()
    tpmOut['ClassMax'] = labels
    tpmOut.loc[tpmOut['Entity'] == 'host', 'ClassMax'] = 'None'

    return tpmOut

In [27]:
TPMmeans

,30,480,Entity,Symbol
Geneid,,,,
gene-SynWH7803_0646,105.179205,134.082216,host,gene-SynWH7803_0646
gene-SynWH7803_1173,125.436432,93.960701,host,gene-SynWH7803_1173
gene-SynWH7803_2450,92.083888,123.054494,host,gene-SynWH7803_2450
gene-SynWH7803_0497,841.042013,628.276443,host,cpeR
gene-SynWH7803_0475,86.077775,102.993276,host,gene-SynWH7803_0475
...,...,...,...,...
gene-SynWH7803_1801,381.062213,413.929476,host,lepA
gene-SynWH7803_1597,36.084155,43.443403,host,gene-SynWH7803_1597
gene-SynWH7803_1516,117.919220,123.454206,host,gene-SynWH7803_1516


In [28]:
TPMmeans = classLabelThreshold(TPMmeans)
TPMmeans = classLabelMax(TPMmeans)

In [29]:
pd.set_option('display.max_rows', 20)
TPMmeans[TPMmeans['Entity'] == 'phage'].sort_index()

,30,480,Entity,Symbol,ClassThreshold,ClassMax
Geneid,,,,,,
gene-SSBP1_gp01,3.413366,30.137439,phage,gene-SSBP1_gp01,early,early
gene-SSBP1_gp02,1.294078,12.575492,phage,gene-SSBP1_gp02,early,early
gene-SSBP1_gp03,5.815364,33.907251,phage,gene-SSBP1_gp03,early,early
gene-SSBP1_gp04,8.192078,44.125760,phage,gene-SSBP1_gp04,early,early
gene-SSBP1_gp05,7.832940,45.670992,phage,gene-SSBP1_gp05,early,early
...,...,...,...,...,...,...
gene-SSBP1_gp51,4.923810,52.952021,phage,gene-SSBP1_gp51,early,early
gene-SSBP1_gp52,2.608220,51.705742,phage,gene-SSBP1_gp52,early,early
gene-SSBP1_gp53,7.156874,37.657970,phage,gene-SSBP1_gp53,early,early


In [30]:
TPMmeans[TPMmeans['Entity'] == 'phage']['ClassMax'].value_counts()

ClassMax
early    55
Name: count, dtype: int64

Add classes to other dfs.

In [31]:
TPMsds[['ClassThreshold', 'ClassMax']] = TPMmeans[['ClassThreshold', 'ClassMax']]
tpms[['ClassThreshold', 'ClassMax']] = TPMmeans[['ClassThreshold', 'ClassMax']]
logs[['ClassThreshold', 'ClassMax']] = TPMmeans[['ClassThreshold', 'ClassMax']]
propExp[['ClassThreshold', 'ClassMax']] = TPMmeans[['ClassThreshold', 'ClassMax']]

# gff3
gff3_final = gff3_genes.merge(TPMmeans[["ClassThreshold", "ClassMax"]], right_index=True, left_on="ID", how="left")
gff3_host  = gff3_final[gff3_final['seq_id'] == 'CT971583.1']
gff3_phage = gff3_final[gff3_final['seq_id'] == 'MT424636.1']

## 6. Add variance to all dataframes

Base variance call on tpms.

In [32]:
def stabilizedVariance(df):
    labels = list()
    
    i = 0
    while i < df.shape[0]:

        # Get array of expression values at time points
        expressions = list(df.iloc[i,0:(df.shape[1]-4)])

        # Get mean expression for the gene
        exprMean = np.mean(np.array(expressions))

        # Get the variance for the gene
        varGene = np.var(np.array(expressions))

        # Stabilized variance
        stableVarGene = varGene/exprMean

        labels.append(stableVarGene)

        i += 1

    tpmOut = df.copy()
    tpmOut['Variance'] = labels

    return tpmOut

In [33]:
tpms = stabilizedVariance(tpms)
tpms

SampleNames,30_R1,480_R1,Entity,Symbol,ClassThreshold,ClassMax,Variance
Geneid,,,,,,,
gene-SynWH7803_0646,105.179205,134.082216,host,gene-SynWH7803_0646,None,None,1.745756
gene-SynWH7803_1173,125.436432,93.960701,host,gene-SynWH7803_1173,None,None,2.257827
gene-SynWH7803_2450,92.083888,123.054494,host,gene-SynWH7803_2450,None,None,2.229213
gene-SynWH7803_0497,841.042013,628.276443,host,cpeR,None,None,15.404825
gene-SynWH7803_0475,86.077775,102.993276,host,gene-SynWH7803_0475,None,None,0.756684
...,...,...,...,...,...,...,...
gene-SynWH7803_1801,381.062213,413.929476,host,lepA,None,None,0.679414
gene-SynWH7803_1597,36.084155,43.443403,host,gene-SynWH7803_1597,None,None,0.340502
gene-SynWH7803_1516,117.919220,123.454206,host,gene-SynWH7803_1516,None,None,0.063462


In [34]:
logs['Variance'] = tpms['Variance']
TPMmeans['Variance'] = tpms['Variance']
TPMsds['Variance'] = tpms['Variance']
propExp['Variance'] = tpms['Variance']

## 7. Write data to output

In [35]:
df_norRNAs[['Entity', 'Symbol']] = tpms[['Entity', 'Symbol']]
df_norRNAs

SampleNames,30_R1,480_R1,Entity,Symbol
Geneid,,,,
gene-SynWH7803_0646,139,152,host,gene-SynWH7803_0646
gene-SynWH7803_1173,500,321,host,gene-SynWH7803_1173
gene-SynWH7803_2450,253,290,host,gene-SynWH7803_2450
gene-SynWH7803_0497,965,618,host,cpeR
gene-SynWH7803_0475,153,157,host,gene-SynWH7803_0475
...,...,...,...,...
gene-SynWH7803_1801,2569,2393,host,lepA
gene-SynWH7803_1597,92,95,host,gene-SynWH7803_1597
gene-SynWH7803_1516,792,711,host,gene-SynWH7803_1516


In [36]:
pd.set_option("display.max_rows", None)
df_norRNAs[df_norRNAs["Entity"] == "phage"]

SampleNames,30_R1,480_R1,Entity,Symbol
Geneid,,,,
gene-SSBP1_gp20,5,120,phage,gene-SSBP1_gp20
gene-SSBP1_gp31,3,89,phage,gene-SSBP1_gp31
gene-SSBP1_gp39,38,470,phage,gene-SSBP1_gp39
gene-SSBP1_gp12,12,167,phage,gene-SSBP1_gp12
gene-SSBP1_gp10,1,23,phage,gene-SSBP1_gp10
gene-SSBP1_gp28,8,77,phage,gene-SSBP1_gp28
gene-SSBP1_gp35,28,331,phage,gene-SSBP1_gp35
gene-SSBP1_gp07,2,31,phage,gene-SSBP1_gp07
gene-SSBP1_gp55,9,48,phage,gene-SSBP1_gp55


In [37]:
# Full TPM table
tpms.to_csv('Huang_ctrl_full_TPM.tsv', sep = '\t')
# Full raw_counts table
df_norRNAs.to_csv('Huang_ctrl_full_raw_counts.tsv', sep = '\t')
# Summarized (time point means) TPM table
TPMmeans.to_csv('Huang_ctrl_TPM_means.tsv', sep = '\t')
# Summarized (time point) TPM standard deviation
TPMsds.to_csv('Huang_ctrl_TPM_std.tsv', sep = '\t')
# Proportional expression per gene and time point
propExp.to_csv('Huang_ctrl_fractional_expression.tsv', sep = '\t')
# Processed gff3 file
gff3_host.to_csv('Huang_ctrl_host_gff3.tsv', sep='\t')
gff3_phage.to_csv('Huang_ctrl_phage_gff3.tsv', sep='\t')